<a href="https://colab.research.google.com/github/akashlogics/prodigy-infotech-intership/blob/main/prodigy_infotech_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **House Prices - Advanced Regression Techniques**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
for dirname, _, filenames in os.walk('/content/data_description.txt'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/train.csv')


In [ ]:
train_id=train_df.Id
test_id=test_df.Id

y = train_df.SalePrice

train_df.isnull().sum()[train_df.isnull().sum()>0].sort_values(ascending=False)
train_df.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu'], axis=1, inplace=True)
test_df.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu'], axis=1, inplace=True)

In [ ]:
cont_var = train_df.select_dtypes(include=[int, float]).columns
cat_var = train_df.select_dtypes(include=object).columns
cont_var = cont_var.drop('SalePrice')

In [ ]:
#Fill missing values for categorical variables
train_df[cont_var] = train_df[cont_var].fillna(train_df[cont_var].mean())
train_df[cat_var] = train_df[cat_var].ffill()


test_df[cont_var] = test_df[cont_var].fillna(test_df[cont_var].mean())
test_df[cat_var] = test_df[cat_var].ffill()

In [ ]:
# Get dummies
X = pd.get_dummies(train_df[cat_var])
X[cont_var] = train_df[cont_var]
test_X = pd.get_dummies(test_df[cat_var])
test_X[cont_var] = test_df[cont_var]

# Create missing columns in test_df from train_df
xoCol = np.setxor1d(X.columns, test_X.columns)
test_X[xoCol] = X[xoCol].ffill()
test_X.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,False,False,False,True,False,False,True,False,False,False,...,548,0,61,0,0,0,0,0,2,2008
1,False,False,False,True,False,False,True,False,False,False,...,460,298,0,0,0,0,0,0,5,2007
2,False,False,False,True,False,False,True,True,False,False,...,608,0,42,0,0,0,0,0,9,2008
3,False,False,False,True,False,False,True,True,False,False,...,642,0,35,272,0,0,0,0,2,2006
4,False,False,False,True,False,False,True,True,False,False,...,836,192,84,0,0,0,0,0,12,2008


In [ ]:
# Sort Columns
X = X.reindex(sorted(X.columns), axis=1)
test_X = test_X.reindex(sorted(test_X.columns), axis=1)
X.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,True,False,False,False,False,False,...,False,True,8,856,True,False,0,2003,2003,2008
1,1262,0,0,3,True,False,False,False,False,False,...,False,True,6,1262,True,False,298,1976,1976,2007
2,920,866,0,3,True,False,False,False,False,False,...,False,True,6,920,True,False,0,2001,2002,2008
3,961,756,0,3,True,False,False,False,False,False,...,False,True,7,756,True,False,0,1915,1970,2006
4,1145,1053,0,4,True,False,False,False,False,False,...,False,True,9,1145,True,False,192,2000,2000,2008


In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# GridSearch to find the best n_estimators and max_depth
params_to_test = {
    'n_estimators':[10,25,50,100],
    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1.0],
    'max_depth':[3,5,7]
}

# GradientBoostingRegressor model
gbr_model = GradientBoostingRegressor(random_state=41)
grid_search = GridSearchCV(gbr_model, param_grid=params_to_test, cv=5, scoring='neg_root_mean_squared_error', n_jobs=4) # Scoring is root mean squared error
grid_search.fit(train_X, train_y)

# Define our model to be the best estimator in the grid search
model = grid_search.best_estimator_

In [ ]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [ ]:
val_pred = model.predict(val_X)
# Root mean squared error, lower is better
mean_squared_error(np.log(val_pred), np.log(val_y), squared=False)

0.12330842977340882

In [ ]:
# Visualization of SalePrice to SalePrediction
sale_predict = model.predict(X)
output = pd.DataFrame({'Id' : train_id, 'SalePrice' : train_df.SalePrice, 'SalePredict' : sale_predict})
output.to_csv('/content/task1 solution.csv', index=False)
output

,Id,SalePrice,SalePredict
0,1,208500,203562.325256
1,2,181500,170476.757840
2,3,223500,206254.901495
3,4,140000,164305.199537
4,5,250000,287460.719656
...,...,...,...
1455,1456,175000,167030.583550
1456,1457,210000,209691.251280
1457,1458,266500,255205.093979
1458,1459,142125,141171.763513


from matplotlib import pyplot as plt
output['Id'].plot(kind='hist', bins=20, title='Id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
output['SalePrice'].plot(kind='hist', bins=20, title='SalePrice')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
output['SalePredict'].plot(kind='hist', bins=20, title='SalePredict')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
output.plot(kind='scatter', x='Id', y='SalePrice', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Id']
  ys = series['SalePredict']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = output.sort_values('Id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Id')
_ = plt.ylabel('SalePredict')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Id']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Id'}, axis=1)
              .sort_values('Id', ascending=True))
  xs = counted['Id']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = output.sort_values('Id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Id')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
output['Id'].plot(kind='line', figsize=(8, 4), title='Id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
output['SalePrice'].plot(kind='line', figsize=(8, 4), title='SalePrice')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
output['SalePredict'].plot(kind='line', figsize=(8, 4), title='SalePredict')
plt.gca().spines[['top', 'right']].set_visible(False)